In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, classification_report, confusion_matrix
import warnings
warnings.filterwarnings('ignore')

# Carregar os dados
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

print("Shape do treino:", train_df.shape)
print("Shape do teste:", test_df.shape)
print("\nPrimeiras linhas do treino:")
print(train_df.head())

Shape do treino: (6736, 10)
Shape do teste: (2164, 9)

Primeiras linhas do treino:
                         ID  Distributor  Product  Duration  Destination  \
0      fffe3800370038003900            7        1        22          122   
1  fffe34003200370037003500            7        1        26           52   
2  fffe32003100320030003200            7       10        15           83   
3  fffe34003400310037003000            8       25        24           55   
4  fffe32003400390038003000            6       16        12          122   

   Sales  Commission  Gender  Age  Target  
0   31.0        0.00     NaN   20       0  
1   22.0        0.00     NaN   36       0  
2   63.0        0.00     NaN   34       0  
3   62.0       24.80     0.0  118       0  
4   19.8       11.88     NaN   26       0  


In [ ]:
# Ver informações dos dados
print(train_df.info())
print("\nValores nulos:")
print(train_df.isnull().sum())

# Ver distribuição do target
print("\nDistribuição do Target:")
print(train_df['Target'].value_counts())
print("\nProporção:")
print(train_df['Target'].value_counts(normalize=True))

# Ver estatísticas
print("\nEstatísticas descritivas:")
print(train_df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6736 entries, 0 to 6735
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           6736 non-null   object 
 1   Distributor  6736 non-null   int64  
 2   Product      6736 non-null   int64  
 3   Duration     6736 non-null   int64  
 4   Destination  6736 non-null   int64  
 5   Sales        6736 non-null   float64
 6   Commission   6736 non-null   float64
 7   Gender       2032 non-null   float64
 8   Age          6736 non-null   int64  
 9   Target       6736 non-null   int64  
dtypes: float64(3), int64(6), object(1)
memory usage: 526.4+ KB
None

Valores nulos:
ID                0
Distributor       0
Product           0
Duration          0
Destination       0
Sales             0
Commission        0
Gender         4704
Age               0
Target            0
dtype: int64

Distribuição do Target:
Target
0    6420
1     316
Name: count, dtype: int64

Proporção:
T

In [ ]:
# Função para processar os dados
def prepare_data(df, is_train=True):
    df = df.copy()

    # Separar ID e Target
    ids = df['ID']
    if is_train:
        target = df['Target']
        df = df.drop(['ID', 'Target'], axis=1)
    else:
        df = df.drop(['ID'], axis=1)

    # Identificar colunas categóricas e numéricas
    categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
    numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()

    print(f"Colunas categóricas: {categorical_cols}")
    print(f"Colunas numéricas: {numerical_cols}")

    # Preencher valores nulos
    for col in numerical_cols:
        if df[col].isnull().sum() > 0:
            df[col].fillna(df[col].median(), inplace=True)

    for col in categorical_cols:
        if df[col].isnull().sum() > 0:
            df[col].fillna(df[col].mode()[0], inplace=True)

    # Label Encoding para variáveis categóricas
    label_encoders = {}
    for col in categorical_cols:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le

    if is_train:
        return df, target, ids
    else:
        return df, ids

# Processar dados de treino e teste
X_train, y_train, train_ids = prepare_data(train_df, is_train=True)
X_test, test_ids = prepare_data(test_df, is_train=False)

print("\nDados processados!")
print("Shape X_train:", X_train.shape)
print("Shape X_test:", X_test.shape)

Colunas categóricas: []
Colunas numéricas: ['Distributor', 'Product', 'Duration', 'Destination', 'Sales', 'Commission', 'Gender', 'Age']
Colunas categóricas: []
Colunas numéricas: ['Distributor', 'Product', 'Duration', 'Destination', 'Sales', 'Commission', 'Gender', 'Age']

Dados processados!
Shape X_train: (6736, 8)
Shape X_test: (2164, 8)


In [ ]:
# Criar e treinar o modelo Random Forest
model = RandomForestClassifier(
    n_estimators=200,
    max_depth=15,
    min_samples_split=10,
    min_samples_leaf=5,
    random_state=42,
    n_jobs=-1,
    class_weight='balanced'  # Importante para classes desbalanceadas
)

print("Treinando o modelo...")
model.fit(X_train, y_train)
print("Modelo treinado!")

Treinando o modelo...
Modelo treinado!


In [ ]:
# Fazer previsões no treino
train_pred = model.predict(X_train)

# Calcular F1-Score conforme especificado no enunciado
train_f1 = f1_score(y_train, train_pred, average='weighted')
train_score = 100 * train_f1  # Multiplicar por 100 conforme enunciado

print(f"\nResultados no conjunto de treino:")
print(f"F1-Score (weighted): {train_f1:.4f}")
print(f"Score final (x100): {train_score:.2f}")

print("\n Matriz de Confusão:")
print(confusion_matrix(y_train, train_pred))

print("\nRelatório de Classificação:")
print(classification_report(y_train, train_pred))


Resultados no conjunto de treino:
F1-Score (weighted): 0.9454
Score final (x100): 94.54

 Matriz de Confusão:
[[6005  415]
 [  30  286]]

Relatório de Classificação:
              precision    recall  f1-score   support

           0       1.00      0.94      0.96      6420
           1       0.41      0.91      0.56       316

    accuracy                           0.93      6736
   macro avg       0.70      0.92      0.76      6736
weighted avg       0.97      0.93      0.95      6736



In [ ]:
# Validação cruzada para verificar a performance
cv_scores = cross_val_score(
    model, X_train, y_train,
    cv=5,
    scoring='f1_weighted',
    n_jobs=-1
)

# Multiplicar por 100 para ter o score final
cv_scores_final = cv_scores * 100

print(f"\n Scores da validação cruzada (5-fold):")
for i, score in enumerate(cv_scores_final, 1):
    print(f"  Fold {i}: {score:.2f}")

print(f"\nMédia: {cv_scores_final.mean():.2f}")
print(f"Desvio padrão: {cv_scores_final.std():.2f}")
print(f"Intervalo: [{cv_scores_final.mean() - cv_scores_final.std():.2f}, {cv_scores_final.mean() + cv_scores_final.std():.2f}]")


 Scores da validação cruzada (5-fold):
  Fold 1: 91.67
  Fold 2: 91.24
  Fold 3: 92.27
  Fold 4: 91.40
  Fold 5: 91.43

Média: 91.60
Desvio padrão: 0.36
Intervalo: [91.24, 91.96]


In [ ]:
# Fazer previsões
test_predictions = model.predict(X_test)

print("\nDistribuição das previsões no teste:")
unique, counts = np.unique(test_predictions, return_counts=True)
for val, count in zip(unique, counts):
    print(f"Classe {val}: {count} ({count/len(test_predictions)*100:.1f}%)")


Distribuição das previsões no teste:
Classe 0: 1953 (90.2%)
Classe 1: 211 (9.8%)


In [ ]:
# Criar DataFrame de submissão
submission = pd.DataFrame({
    'ID': test_ids,
    'Target': test_predictions
})

# Salvar arquivo
submission.to_csv('sample_submission.csv', index=False)

print("\n Arquivo 'sample_submission.csv' criado com sucesso!")
print(f"Total de previsões: {len(submission)}")




 Arquivo 'sample_submission.csv' criado com sucesso!
Total de previsões: 2164


In [ ]:
print(submission.head(10))

                         ID  Target
0  fffe31003600330038003500       0
1  fffe33003600300031003400       1
2          fffe320033003300       1
3          fffe390039003800       0
4      fffe3500350031003000       0
5  fffe31003000300037003300       0
6  fffe33003300360037003200       0
7  fffe32003500310030003200       0
8  fffe32003100320031003600       0
9      fffe3900380030003200       0
